In [ ]:
import numpy as np
import pyupbit
import datetime
import time
import matplotlib.pyplot as plt
from joblib import load
from sklearn.preprocessing import minmax_scale
from keras.models import save_model

In [ ]:
# 본인의 업비트 access 와 secret key를 적어주세요
access = "본인의 access를 입력하세요"
secret = "본인의 secret key를 입력하세요"

upbit = pyupbit.Upbit(access, secret)

In [ ]:
def get_train_price_data(ticker): ### 05/22 19:58 ###
    df = pyupbit.get_ohlcv(ticker, interval="minute1", count=116295)
    # 데이터프레임, 현재가 반환
    return df

In [ ]:
df=get_train_price_data("KRW-ONG")

In [ ]:
df=df.T
df

In [ ]:
array=df.to_numpy()
array.shape

In [ ]:
A = []
for i in range(0,116295-1394,15) :
    obs=array[:,i:(i+1395)]
    obs=minmax_scale(obs, axis=1, copy=True)
    A.append(obs)
array_scaled=np.array(A)

In [ ]:
array_scaled.shape

In [ ]:
x_train=array_scaled[:,:,:-15]
x_train.shape

In [ ]:
y_train=array_scaled[:,0,-15:]
y_train.shape

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

model=Sequential()
model.add(LSTM(120, input_shape=(6, 1380), dropout=0.1, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(120, dropout=0.1, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(120, dropout=0.1, recurrent_dropout=0.2))
model.add(Dense(15, activation='linear'))

model.compile(loss='mae', optimizer='adam')

# 모델의 실행
history = model.fit(x_train, y_train, epochs=100, batch_size=64, verbose=1)

model.summary()

In [ ]:
save_model('./model/lstm.h5', model)